In [33]:
from google.colab import drive
drive.mount('/content/drive')

basic_path = '/content/drive/MyDrive/test/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
# 라이브러리 import
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

In [35]:
def get_device():
    return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
def get_mean_std(channel, training_dataset):
  if channel.lower() is 'rgb':
    mean_rgb = [np.mean(x.numpy(), axis=(1, 2)) for x,_ in training_dataset] # (1, 2) 기준으로 평균값 계산. size: 6,0000
    std_rgb = [np.std(x.numpy(), axis=(1, 2)) for x,_ in training_dataset] # 표준편차 계산
    
#    예시
#     >>> import numpy as np
#     >>> arr = [ [[1, 2, 3], [3, 4, 5], [6, 7, 8]], [[1, 2, 3], [3, 1, 5], [6, 7, 8]], [[1, 2, 3], [3, 7, 5], [6, 7, 8]] ]
#     >>> np_arr = np.asarray(arr)
#     >>> np_arr.shape
#     (3, 3, 3)
#     >>> np.mean(np_arr, axis=(1, 2))
#     array([4.33333333, 4.        , 4.66666667])
#
#     (1 + 2 + 3 + 3 + 4 + 5 + 6 + 7 + 8) / 9. (1 + 2 + 3 + 3 + 1 + 5 + 6 + 7 + 8) / 9, (1 + 2 + 3 + 3 + 7 + 5 + 6 + 7 + 8) / 9

    mean_r = np.mean([m[0] for m in mean_rgb])
    mean_g = np.mean([m[1] for m in mean_rgb])
    mean_b = np.mean([m[2] for m in mean_rgb])

    std_r = np.mean([s[0] for s in std_rgb])
    std_g = np.mean([s[1] for s in std_rgb])
    std_b = np.mean([s[2] for s in std_rgb])
    return [mean_r, mean_g, mean_b], [std_r, std_g, std_b]
  else:
    mean = [np.mean(x.numpy(), axis=(1, 2)) for x,_ in training_dataset]
    std = [np.std(x.numpy(), axis=(1, 2)) for x,_ in training_dataset]

    return np.mean([m[0] for m in mean]), np.mean([s[0] for s in std])

### ResNet 구현

In [37]:
class BottleNeckBlock(nn.Module):
  def __init__(self, input_channel_size, output_channel_size, stride):
    super(BottleNeckBlock, self).__init__()
    self.residual_func = nn.Sequential(
            nn.Conv2d(in_channels=input_channel_size, out_channels=output_channel_size, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(output_channel_size),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=output_channel_size, out_channels=output_channel_size, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(output_channel_size),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=output_channel_size, out_channels=(output_channel_size * 4), kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(output_channel_size * 4),
        )
    self.myReLU = nn.ReLU()
    self.shortcut = nn.Sequential()
    
    # 일부 Layer들을 건너뛰기 위함. stride가 1이 아니거나, input_channel이 마지막 layer의 output_size와 같지 않을 때 추가
    if (stride != 1) or (input_channel_size != (output_channel_size * 4)):
      self.shortcut = nn.Sequential(
            nn.Conv2d(input_channel_size, (output_channel_size * 4), kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(output_channel_size * 4)
        )
  
  def forward(self, x):
    x = self.residual_func(x) + self.shortcut(x)
    x = self.myReLU(x)
    return x

In [38]:
class MyResNet(nn.Module):
  def __init__(self, size_of_channel, number_of_class):
    super(MyResNet, self).__init__()
    self.base_input_channel = 64
    self.conv1 = nn.Sequential(
        nn.Conv2d(size_of_channel, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
    )
    # ResNet 50
    self.conv2 = self.__generate_bottleneck_layer__(64, number_of_layer=3, base_stride=1)
    self.conv3 = self.__generate_bottleneck_layer__(128, number_of_layer=4)
    self.conv4 = self.__generate_bottleneck_layer__(256, number_of_layer=6)
    self.conv5 = self.__generate_bottleneck_layer__(512, number_of_layer=3)

    self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc = nn.Linear(2048, number_of_class)

    self.__init_weights__()

  def __generate_bottleneck_layer__(self, output_channel_size, number_of_layer, base_stride=2):
    strides = [base_stride] + [1] * (number_of_layer - 1)
    layers = []
    for stride in strides:
        layer = BottleNeckBlock(self.base_input_channel, output_channel_size, stride)
        self.base_input_channel = output_channel_size * 4
        layers.append(layer)
    return nn.Sequential(*layers)
  
  def __init_weights__(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
          nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.constant_(m.bias, 0)

  def forward(self, x):
    output = self.conv1(x)
    output = self.conv2(output)
    x = self.conv3(output)
    x = self.conv4(x)
    x = self.conv5(x)
    x = self.avg_pool(x)
    x = x.view(x.size(0), -1) # Tensor의 차원을 1차원으로 변경
    x = self.fc(x)
    return x

### Train / Test 코드

In [39]:
def train(device, dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      
      pred = model(X.cuda())
      loss = loss_fn(pred, y)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      print('.', end='')
      if batch % 100 == 0:
          print()
          loss, current = loss.item(), batch*len(X)
          print(f'loss: {loss:>7f}   [{current:>5d}/{size:>5d}]')

In [40]:
def test(device, dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loass: {test_loss:>8f}\n')

### Model - Fashion MNIST

In [41]:
device = get_device()
print('Device:', device)

Device: cuda


In [42]:
training_dataset_fashion = datasets.FashionMNIST(
    root=basic_path + '/data', train=True, download=True, transform=transforms.ToTensor(),
)
test_dataset_fashion = datasets.FashionMNIST(
    root=basic_path + '/data', train=False, download=True, transform=transforms.ToTensor(),
)

gray_scale_mean, gray_scale_std = get_mean_std('gray', training_dataset_fashion)
gray_scale_transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Resize(224),
  transforms.Normalize(mean=[gray_scale_mean], std=[gray_scale_std])
])
training_dataset_fashion.transform = gray_scale_transform
test_dataset_fashion.transform = gray_scale_transform

In [43]:
training_dataloader_fashion = DataLoader(training_dataset_fashion, batch_size=64)
test_dataloader_fashion = DataLoader(test_dataset_fashion, batch_size=64)

for X, y in test_dataloader_fashion:
  print('Shape of X [N, C, H, W]:', X.shape)
  print('Shape of y:', y.shape, y.dtype)
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 224, 224])
Shape of y: torch.Size([64]) torch.int64


In [44]:
model_fashion = MyResNet(1, len(datasets.FashionMNIST.classes))
model_fashion = model_fashion.to(device)

In [45]:
# Cuda out of memory
import gc
gc.collect()
torch.cuda.empty_cache()

In [46]:
from torchsummary import summary

summary(model_fashion, input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           3,136
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [47]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [48]:
for epoch in range(5):
  print(f'Epoch {epoch + 1}\n---------------------------------')
  train(device, training_dataloader_fashion, model_fashion, nn.CrossEntropyLoss(reduction='sum'),
        torch.optim.Adam(model_fashion.parameters(),lr=0.001))
  test(device, test_dataloader_fashion, model_fashion, nn.CrossEntropyLoss(reduction='sum'))

Epoch 1
---------------------------------
.
loss: 154.193176   [    0/60000]
....................................................................................................
loss: 37.031403   [ 6400/60000]
....................................................................................................
loss: 22.839159   [12800/60000]
....................................................................................................
loss: 31.285709   [19200/60000]
....................................................................................................
loss: 19.671614   [25600/60000]
....................................................................................................
loss: 28.005474   [32000/60000]
....................................................................................................
loss: 16.552950   [38400/60000]
....................................................................................................
loss: 29.169474   [44800

In [54]:
dest_model_path = basic_path + 'resnet_models/model_fashion.pth'
print(dest_model_path)
torch.save(model_fashion.state_dict(), dest_model_path)

/content/drive/MyDrive/test/resnet_models/model_fashion.pth
